# Multi-Agent Workflows with Builder and Flow

Learn how to build sophisticated multi-agent workflows using lionpride's Builder and Flow system. This notebook demonstrates how to create operation graphs where different agents collaborate, specialize in tasks, and execute in dependency-aware order.

## Setup

First, let's import the necessary components and set up our environment.

In [ ]:
import os

from pydantic import BaseModel, Field

from lionpride import Session
from lionpride.ipu import IPU
from lionpride.operations import Builder, flow, flow_stream
from lionpride.services import iModel

# Ensure API keys are set
assert os.getenv("OPENAI_API_KEY"), "Set OPENAI_API_KEY environment variable"
print("✓ Environment ready")

## 1. Why Multi-Agent Workflows?

### The Problem: Complex Task Decomposition

Real-world tasks often require:
- **Specialization**: Different expertise for different subtasks (research vs. writing vs. editing)
- **Parallel Processing**: Multiple independent analyses that can run simultaneously
- **Sequential Dependencies**: Results that build on previous outputs
- **Coordination**: Aggregating results from multiple sources

### The Solution: Builder + Flow + IPU

lionpride's Builder and Flow system provides:
1. **Builder**: Declarative operation graph construction with dependency tracking
2. **Flow**: Automatic execution ordering based on dependencies
3. **IPU**: Validated execution context with session management
4. **Parallelization**: Concurrent execution of independent operations

### Architecture

```
Session + IPU
  ├── Services (iModel instances)
  └── Builder
       ├── Operations via add("name", "type", params, depends_on=[...])
       └── build() → Graph (DAG)
            ↓
         flow(session, branch, graph, ipu) → Execution
```

## 2. Builder Basics

The `Builder` class helps you construct operation graphs. Each operation represents an agent task, and dependencies define execution order.

### Key Concepts

- **Operations**: Added via `builder.add(name, operation_type, parameters, depends_on=[...])`
- **Dependencies**: `depends_on=["op1", "op2"]` specifies which operations must complete first
- **Branches**: Logical grouping of messages in the session
- **Graph**: `builder.build()` returns the operation graph
- **Execution**: `flow(session, branch, graph, ipu)` executes in optimal order

In [ ]:
# Basic builder setup
async def builder_basics():
    """Demonstrate basic Builder pattern"""
    # Create session and IPU
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    # Register a model
    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7, name="assistant")
    session.services.register(model)

    # Create branch for conversation
    branch = session.create_branch(name="main")

    # Create builder and add operation using builder.add()
    builder = Builder()
    builder.add(
        "explain_graph",  # Operation name
        "communicate",  # Operation type
        {
            "instruction": "Explain what an operation graph is in one sentence",
            "imodel": "assistant",
        },
    )

    # Build the graph and execute with flow
    graph = builder.build()
    results = await flow(session, branch, graph, ipu)

    print("Operation Result:")
    print(results["explain_graph"])
    return results


# Run the example
await builder_basics()

## 3. Sequential Operations (A → B → C)

Sequential workflows ensure operations execute in a specific order. Each operation depends on the previous one's output.

### Use Case: Research → Write → Edit Pipeline

This is a classic content creation workflow:
1. **Research**: Gather facts and information
2. **Write**: Create initial draft using research
3. **Edit**: Refine and polish the draft

In [ ]:
async def sequential_workflow():
    """Sequential workflow: research → write → edit"""
    # Setup session and IPU
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    # Create specialized models for each role
    researcher = iModel(
        provider="openai",
        model="gpt-4o-mini",
        temperature=0.3,  # More factual
        name="researcher",
    )

    writer = iModel(
        provider="openai",
        model="gpt-4o-mini",
        temperature=0.7,  # More creative
        name="writer",
    )

    editor = iModel(
        provider="openai",
        model="gpt-4o-mini",
        temperature=0.2,  # More precise
        name="editor",
    )

    for model in [researcher, writer, editor]:
        session.services.register(model)

    # Create branch
    branch = session.create_branch(name="content-pipeline")

    # Build operation graph using builder.add()
    builder = Builder()

    # Step 1: Research (no dependencies)
    builder.add(
        "research",
        "communicate",
        {
            "instruction": "Research key facts about quantum computing for a blog post. Focus on: qubits, superposition, and practical applications.",
            "imodel": "researcher",
        },
    )

    # Step 2: Write (depends on research)
    builder.add(
        "write",
        "communicate",
        {
            "instruction": (
                "Using the research provided, write a 200-word blog post "
                "about quantum computing for a general audience. Make it engaging and accessible."
            ),
            "imodel": "writer",
        },
        depends_on=["research"],  # Waits for research to complete
    )

    # Step 3: Edit (depends on writing)
    builder.add(
        "edit",
        "communicate",
        {
            "instruction": "Edit this blog post for clarity, engagement, and flow. Make it shine!",
            "imodel": "editor",
        },
        depends_on=["write"],  # Waits for writing to complete
    )

    # Build and execute workflow
    graph = builder.build()
    print("🔄 Executing sequential workflow...\n")
    results = await flow(session, branch, graph, ipu)

    print("" + "=" * 60)
    print("📚 RESEARCH OUTPUT")
    print("=" * 60)
    print(results["research"])

    print("\n" + "=" * 60)
    print("✍️  DRAFT OUTPUT")
    print("=" * 60)
    print(results["write"])

    print("\n" + "=" * 60)
    print("✨ FINAL EDITED OUTPUT")
    print("=" * 60)
    print(results["edit"])

    return results


# Run the sequential workflow
await sequential_workflow()

### Key Observations

1. **Explicit Dependencies**: `context_from=[previous_op]` ensures proper ordering
2. **Context Propagation**: Each agent sees the results from previous steps
3. **Specialization**: Different temperature settings optimize for different tasks
4. **Automatic Ordering**: `flow()` executes operations in dependency order

## 4. Parallel Operations (A, B, C Simultaneously)

When operations don't depend on each other, they can run in parallel. This significantly reduces total execution time.

### Use Case: Multi-Perspective Analysis

Analyze a business decision from multiple independent perspectives simultaneously:
- HR perspective
- Finance perspective
- Operations perspective

Then synthesize all analyses into a final recommendation.

In [ ]:
# Define structured output for analyses
class Analysis(BaseModel):
    """Structured analysis result"""

    perspective: str = Field(description="The perspective being analyzed from")
    key_points: list[str] = Field(description="Key findings (3-5 points)")
    recommendation: str = Field(description="Specific recommendation")
    confidence: float = Field(ge=0, le=1, description="Confidence in recommendation")


async def parallel_workflow():
    """Parallel analysis workflow with synthesis"""
    # Setup
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="parallel-analysis")
    topic = "Should our company adopt a 4-day work week?"

    builder = Builder()

    # Parallel analyses from different perspectives (no dependencies between them)
    builder.add(
        "hr_analysis",
        "communicate",
        {
            "instruction": f"Analyze from HR/People perspective: {topic}",
            "imodel": model.name,
            "response_model": Analysis,
        },
    )

    builder.add(
        "finance_analysis",
        "communicate",
        {
            "instruction": f"Analyze from financial/cost perspective: {topic}",
            "imodel": model.name,
            "response_model": Analysis,
        },
    )

    builder.add(
        "operations_analysis",
        "communicate",
        {
            "instruction": f"Analyze from operations/logistics perspective: {topic}",
            "imodel": model.name,
            "response_model": Analysis,
        },
    )

    # Synthesis depends on ALL three analyses
    builder.add(
        "synthesis",
        "communicate",
        {
            "instruction": (
                "Synthesize the HR, finance, and operations analyses into "
                "a balanced recommendation for leadership. Weigh all perspectives "
                "and provide a clear go/no-go decision with rationale."
            ),
            "imodel": model.name,
        },
        depends_on=["hr_analysis", "finance_analysis", "operations_analysis"],
    )

    # Build and execute
    graph = builder.build()
    print("🔄 Executing parallel workflow...\n")
    print("⚡ HR, Finance, and Operations analyses running in parallel...\n")

    results = await flow(session, branch, graph, ipu)

    # Display results
    print("=" * 60)
    print("👥 HR ANALYSIS")
    print("=" * 60)
    hr = results["hr_analysis"]
    if isinstance(hr, Analysis):
        print(f"Perspective: {hr.perspective}")
        print("Key Points:")
        for point in hr.key_points:
            print(f"  • {point}")
        print(f"Recommendation: {hr.recommendation}")
        print(f"Confidence: {hr.confidence:.0%}")
    else:
        print(hr)

    print("\n" + "=" * 60)
    print("💰 FINANCE ANALYSIS")
    print("=" * 60)
    fin = results["finance_analysis"]
    if isinstance(fin, Analysis):
        print(f"Perspective: {fin.perspective}")
        print("Key Points:")
        for point in fin.key_points:
            print(f"  • {point}")
        print(f"Recommendation: {fin.recommendation}")
        print(f"Confidence: {fin.confidence:.0%}")
    else:
        print(fin)

    print("\n" + "=" * 60)
    print("⚙️  OPERATIONS ANALYSIS")
    print("=" * 60)
    ops = results["operations_analysis"]
    if isinstance(ops, Analysis):
        print(f"Perspective: {ops.perspective}")
        print("Key Points:")
        for point in ops.key_points:
            print(f"  • {point}")
        print(f"Recommendation: {ops.recommendation}")
        print(f"Confidence: {ops.confidence:.0%}")
    else:
        print(ops)

    print("\n" + "=" * 60)
    print("🎯 FINAL SYNTHESIS")
    print("=" * 60)
    print(results["synthesis"])

    return results


# Run the parallel workflow
await parallel_workflow()

### Performance Benefits

**Sequential execution**: `T_total = T_hr + T_finance + T_ops + T_synthesis`

**Parallel execution**: `T_total = max(T_hr, T_finance, T_ops) + T_synthesis`

If each analysis takes ~10 seconds:
- Sequential: 40 seconds total
- Parallel: ~20 seconds total (50% time reduction!)

## 5. Dependencies and Data Flow

Understanding how data flows through the operation graph is crucial.

### Dependency Patterns

```python
# No dependencies → Parallel execution
builder.add("op_a", "communicate", {...})
builder.add("op_b", "communicate", {...})
# A and B run simultaneously

# Linear dependency → Sequential execution
builder.add("op_a", "communicate", {...})
builder.add("op_b", "communicate", {...}, depends_on=["op_a"])
# B waits for A

# Diamond dependency → Mixed execution
builder.add("op_a", "communicate", {...})
builder.add("op_b", "communicate", {...}, depends_on=["op_a"])
builder.add("op_c", "communicate", {...}, depends_on=["op_a"])
builder.add("op_d", "communicate", {...}, depends_on=["op_b", "op_c"])
# A → (B, C in parallel) → D
```

### Data Flow Visualization

```
Sequential:    A → B → C → D

Parallel:      A ⎫
               B ⎬ → D
               C ⎭

Diamond:       A
              ↙ ↘
             B   C
              ↘ ↙
               D
```

In [ ]:
async def dependency_demo():
    """Demonstrate different dependency patterns"""
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="diamond")
    builder = Builder()

    # Diamond pattern: A → (B, C in parallel) → D
    print("Building diamond dependency graph:\n")
    print("       A")
    print("      ↙ ↘")
    print("     B   C")
    print("      ↘ ↙")
    print("       D\n")

    # A: Initial task
    builder.add(
        "op_a",
        "communicate",
        {
            "instruction": "Generate 3 random topics for blog posts",
            "imodel": model.name,
        },
    )

    # B and C: Parallel processing of A's output
    builder.add(
        "op_b",
        "communicate",
        {
            "instruction": "Take the topics and suggest target audiences for each",
            "imodel": model.name,
        },
        depends_on=["op_a"],
    )

    builder.add(
        "op_c",
        "communicate",
        {
            "instruction": "Take the topics and suggest keywords for SEO for each",
            "imodel": model.name,
        },
        depends_on=["op_a"],
    )

    # D: Synthesis of B and C
    builder.add(
        "op_d",
        "communicate",
        {
            "instruction": "Combine the audience and keyword analyses into a content strategy",
            "imodel": model.name,
        },
        depends_on=["op_b", "op_c"],
    )

    graph = builder.build()
    print("🔄 Executing diamond workflow...\n")
    results = await flow(session, branch, graph, ipu)

    print("=" * 60)
    print("A: TOPICS")
    print("=" * 60)
    print(results["op_a"])

    print("\n" + "=" * 60)
    print("B: AUDIENCES (parallel with C)")
    print("=" * 60)
    print(results["op_b"])

    print("\n" + "=" * 60)
    print("C: KEYWORDS (parallel with B)")
    print("=" * 60)
    print(results["op_c"])

    print("\n" + "=" * 60)
    print("D: STRATEGY (depends on B and C)")
    print("=" * 60)
    print(results["op_d"])

    return results


await dependency_demo()

## 6. Full Example: Research → Analyze → Summarize Pipeline

Let's build a realistic pipeline that combines sequential and parallel patterns.

### Scenario: Competitive Analysis Workflow

1. **Research**: Gather information about competitors
2. **Parallel Analysis**: Analyze pricing, features, and market positioning simultaneously
3. **Synthesis**: Combine all analyses into strategic recommendations

This demonstrates:
- Structured outputs with Pydantic
- Mixed sequential and parallel execution
- Real-world business use case

In [ ]:
# Define structured outputs
class CompetitorInfo(BaseModel):
    """Basic competitor information"""

    competitors: list[str] = Field(description="List of main competitors")
    market_segment: str = Field(description="Market segment/category")
    key_trends: list[str] = Field(description="Key market trends")


class PricingAnalysis(BaseModel):
    """Pricing analysis"""

    price_ranges: dict[str, str] = Field(description="Price ranges by competitor")
    pricing_model: str = Field(description="Common pricing model")
    insights: list[str] = Field(description="Key pricing insights")


class FeatureAnalysis(BaseModel):
    """Feature analysis"""

    common_features: list[str] = Field(description="Features all competitors have")
    differentiators: list[str] = Field(description="Unique features by competitor")
    gaps: list[str] = Field(description="Missing features/opportunities")


class MarketAnalysis(BaseModel):
    """Market positioning analysis"""

    positioning: dict[str, str] = Field(description="How each competitor positions themselves")
    target_audiences: list[str] = Field(description="Target customer segments")
    market_share_insights: str = Field(description="Market share observations")


async def competitive_analysis_pipeline():
    """Full competitive analysis workflow"""
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.5)
    session.services.register(model)

    branch = session.create_branch(name="competitive")
    product_category = "Project management software"

    builder = Builder()

    # Step 1: Research competitors
    builder.add(
        "research",
        "communicate",
        {
            "instruction": (
                f"Research the competitive landscape for {product_category}. "
                f"Identify the top 5 competitors, the market segment, and key trends."
            ),
            "imodel": model.name,
            "response_model": CompetitorInfo,
        },
    )

    # Step 2: Parallel analyses (all depend on research)
    builder.add(
        "pricing",
        "communicate",
        {
            "instruction": (
                "Analyze the pricing strategies of the competitors identified. "
                "Include price ranges, pricing models, and key insights."
            ),
            "imodel": model.name,
            "response_model": PricingAnalysis,
        },
        depends_on=["research"],
    )

    builder.add(
        "features",
        "communicate",
        {
            "instruction": (
                "Analyze the features offered by the competitors. "
                "Identify common features, differentiators, and gaps in the market."
            ),
            "imodel": model.name,
            "response_model": FeatureAnalysis,
        },
        depends_on=["research"],
    )

    builder.add(
        "market",
        "communicate",
        {
            "instruction": (
                "Analyze the market positioning of the competitors. "
                "How does each position themselves? What audiences do they target?"
            ),
            "imodel": model.name,
            "response_model": MarketAnalysis,
        },
        depends_on=["research"],
    )

    # Step 3: Strategic synthesis
    builder.add(
        "synthesis",
        "communicate",
        {
            "instruction": (
                "Based on the pricing, feature, and market analyses, provide strategic "
                "recommendations for entering this market. Include: "
                "1. Recommended pricing strategy "
                "2. Must-have features vs. differentiators "
                "3. Target market positioning "
                "4. Go-to-market approach"
            ),
            "imodel": model.name,
        },
        depends_on=["pricing", "features", "market"],
    )

    graph = builder.build()
    print(f"🔄 Executing competitive analysis pipeline for: {product_category}\n")
    print("Workflow structure:")
    print("  Research")
    print("     ↓")
    print("  ⎡ Pricing ⎤")
    print("  ⎢ Features ⎥ (parallel)")
    print("  ⎣ Market  ⎦")
    print("     ↓")
    print("  Synthesis\n")

    results = await flow(session, branch, graph, ipu)

    # Display results
    print("=" * 60)
    print("🔍 RESEARCH: COMPETITOR LANDSCAPE")
    print("=" * 60)
    research = results["research"]
    if isinstance(research, CompetitorInfo):
        print(f"Market Segment: {research.market_segment}")
        print("\nCompetitors:")
        for comp in research.competitors:
            print(f"  • {comp}")
        print("\nKey Trends:")
        for trend in research.key_trends:
            print(f"  • {trend}")
    else:
        print(research)

    print("\n" + "=" * 60)
    print("💰 PRICING ANALYSIS")
    print("=" * 60)
    pricing = results["pricing"]
    if isinstance(pricing, PricingAnalysis):
        print(f"Pricing Model: {pricing.pricing_model}")
        print("\nPrice Ranges:")
        for comp, price in pricing.price_ranges.items():
            print(f"  • {comp}: {price}")
        print("\nKey Insights:")
        for insight in pricing.insights:
            print(f"  • {insight}")
    else:
        print(pricing)

    print("\n" + "=" * 60)
    print("⚙️  FEATURE ANALYSIS")
    print("=" * 60)
    features = results["features"]
    if isinstance(features, FeatureAnalysis):
        print("Common Features:")
        for feature in features.common_features:
            print(f"  • {feature}")
        print("\nDifferentiators:")
        for diff in features.differentiators:
            print(f"  • {diff}")
        print("\nMarket Gaps (Opportunities):")
        for gap in features.gaps:
            print(f"  • {gap}")
    else:
        print(features)

    print("\n" + "=" * 60)
    print("🎯 MARKET POSITIONING")
    print("=" * 60)
    market = results["market"]
    if isinstance(market, MarketAnalysis):
        print("Positioning by Competitor:")
        for comp, pos in market.positioning.items():
            print(f"  • {comp}: {pos}")
        print("\nTarget Audiences:")
        for audience in market.target_audiences:
            print(f"  • {audience}")
        print(f"\nMarket Share: {market.market_share_insights}")
    else:
        print(market)

    print("\n" + "=" * 60)
    print("🎯 STRATEGIC RECOMMENDATIONS")
    print("=" * 60)
    print(results["synthesis"])

    return results


# Run the full pipeline
await competitive_analysis_pipeline()

### Pipeline Benefits

1. **Type Safety**: Pydantic models ensure structured, validated outputs
2. **Parallelization**: Pricing, features, and market analyses run concurrently
3. **Specialization**: Each operation focuses on a specific analytical dimension
4. **Composition**: Final synthesis combines insights from all analyses
5. **Maintainability**: Clear dependency graph makes workflow logic explicit

## 7. Streaming Results from Flow

For long-running workflows, you can stream results as operations complete instead of waiting for everything to finish.

### Why Stream?

- **Faster feedback**: See results as they arrive
- **Progress monitoring**: Track workflow execution in real-time
- **Early decisions**: Act on partial results before full completion

### Using `flow_stream()`

`flow_stream()` yields `(operation_id, chunk)` pairs as operations complete.

In [ ]:
async def streaming_workflow():
    """Demonstrate streaming results from workflow"""
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="streaming")
    builder = Builder()

    # Create a simple parallel workflow
    builder.add(
        "poem",
        "communicate",
        {
            "instruction": "Write a short poem about artificial intelligence",
            "imodel": model.name,
        },
    )

    builder.add(
        "story",
        "communicate",
        {
            "instruction": "Write a short story (3 sentences) about a robot learning to dream",
            "imodel": model.name,
        },
    )

    builder.add(
        "haiku",
        "communicate",
        {
            "instruction": "Write a haiku about code",
            "imodel": model.name,
        },
    )

    graph = builder.build()
    print("🔄 Streaming workflow execution...\n")
    print("Watch as results stream in from parallel operations:\n")
    print("=" * 60)

    # Stream results as they complete using flow_stream
    async for result in flow_stream(session, branch, graph, ipu):
        print(f"\n[{result.name.upper()}] ({result.completed}/{result.total} complete)")
        if result.success:
            print(result.result)
        else:
            print(f"Error: {result.error}")

    print("\n" + "=" * 60)
    print("✓ All operations completed!")


# Run the streaming workflow
await streaming_workflow()

### Streaming Best Practices

1. **Enable streaming on the model**: Set `stream=True` when creating iModel
2. **Handle partial results**: Chunks may not be complete sentences
3. **Track operation context**: Use operation IDs to distinguish between parallel streams
4. **Consider buffering**: For UI updates, buffer chunks to avoid excessive renders

## 8. Advanced Patterns

### Error Handling

Operations can fail. Always check execution status in production workflows.

In [ ]:
async def error_handling_demo():
    """Demonstrate error handling in workflows"""
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.7)
    session.services.register(model)

    branch = session.create_branch(name="errors")
    builder = Builder()

    # Create some operations
    builder.add(
        "greeting",
        "communicate",
        {
            "instruction": "Say hello in a friendly way",
            "imodel": model.name,
        },
    )

    graph = builder.build()

    # Execute and check results
    try:
        results = await flow(session, branch, graph, ipu, stop_on_error=True)

        for name, result in results.items():
            print(f"✓ Operation '{name}' succeeded")
            print(f"  Result: {result}")

    except Exception as e:
        print(f"❌ Workflow failed: {e}")


await error_handling_demo()

### Conditional Workflows

You can execute conditional workflows by running `flow()` multiple times based on results.

In [ ]:
class SentimentResult(BaseModel):
    """Sentiment analysis result"""

    sentiment: str = Field(description="positive, negative, or neutral")
    confidence: float = Field(ge=0, le=1)
    needs_escalation: bool = Field(description="Whether human review is needed")


async def conditional_workflow():
    """Conditional workflow based on sentiment analysis"""
    session = Session()
    ipu = IPU()
    ipu.register_session(session)

    model = iModel(provider="openai", model="gpt-4o-mini", temperature=0.3)
    session.services.register(model)

    customer_feedback = (
        "The product quality is terrible and customer service never responded "
        "to my emails. This is the worst experience I've ever had."
    )

    branch = session.create_branch(name="conditional")

    # Step 1: Analyze sentiment
    builder1 = Builder()
    builder1.add(
        "sentiment",
        "communicate",
        {
            "instruction": f"Analyze the sentiment of this customer feedback: {customer_feedback}",
            "imodel": model.name,
            "response_model": SentimentResult,
        },
    )

    graph1 = builder1.build()
    results = await flow(session, branch, graph1, ipu)
    sentiment = results["sentiment"]

    print("=" * 60)
    print("SENTIMENT ANALYSIS")
    print("=" * 60)
    if isinstance(sentiment, SentimentResult):
        print(f"Sentiment: {sentiment.sentiment}")
        print(f"Confidence: {sentiment.confidence:.0%}")
        print(f"Needs Escalation: {sentiment.needs_escalation}")
        needs_escalation = sentiment.needs_escalation
    else:
        print(sentiment)
        needs_escalation = True  # Assume escalation if we can't parse

    # Step 2: Conditional processing
    if needs_escalation:
        print("\n⚠️  Negative sentiment detected - generating escalation\n")

        builder2 = Builder()
        builder2.add(
            "escalation",
            "communicate",
            {
                "instruction": (
                    "Generate a professional escalation email to the customer success team "
                    f"about this negative feedback: {customer_feedback}. Include suggested actions."
                ),
                "imodel": model.name,
            },
        )

        graph2 = builder2.build()
        escalation_results = await flow(session, branch, graph2, ipu)

        print("=" * 60)
        print("ESCALATION EMAIL")
        print("=" * 60)
        print(escalation_results["escalation"])
    else:
        print("\n✓ Sentiment is positive - no escalation needed")


await conditional_workflow()

## 9. Summary

### Key Takeaways

1. **Builder Pattern**: Declarative operation graph construction
   ```python
   builder = Builder()
   builder.add("name", "communicate", {"instruction": "...", "imodel": "..."}, depends_on=[...])
   graph = builder.build()
   ```

2. **IPU**: Validated execution context
   ```python
   ipu = IPU()
   ipu.register_session(session)
   ```

3. **Execution**: `flow()` analyzes dependencies and executes optimally
   ```python
   results = await flow(session, branch, graph, ipu)
   ```

4. **Parallelization**: Operations without dependencies run concurrently
   - Sequential: `A → B → C`
   - Parallel: `A, B, C → D`
   - Diamond: `A → (B, C) → D`

5. **Type Safety**: Use Pydantic models for structured outputs
   ```python
   builder.add("analyze", "communicate", {..., "response_model": MyModel})
   ```

6. **Streaming**: Use `flow_stream()` for real-time progress
   ```python
   async for result in flow_stream(session, branch, graph, ipu):
       print(f"{result.name}: {result.result}")
   ```

### Common Pitfalls to Avoid

1. ❌ **Circular dependencies**: `A → B → A` creates deadlock
2. ❌ **Forgetting await**: `flow()` is async, must use `await`
3. ❌ **Missing IPU registration**: Always `ipu.register_session(session)`
4. ❌ **Wrong API**: Use `builder.add()`, not `builder.communicate()` (doesn't exist)
5. ❌ **Ignoring errors**: Check results or use `stop_on_error=True`

### When to Use Multi-Agent Workflows

✅ **Good use cases**:
- Task decomposition (research → write → edit)
- Multi-perspective analysis (HR, finance, ops)
- Parallel independent tasks
- Complex dependencies

❌ **Not needed for**:
- Single-step tasks
- Simple sequential processing
- No dependency management needed

### Next Steps

- **Tool-calling agents**: Combine workflows with tool usage
- **Iterative refinement**: Implement critique-revise loops
- **Error recovery**: Add retry logic and fallback strategies
- **Production deployment**: Scale workflows with proper monitoring

## Additional Resources

- **API Documentation**: lionpride operations API reference
- **Cookbook**: Multi-agent workflow patterns
- **GitHub**: Example workflows and templates
- **Community**: Share your workflow patterns

---

**Happy orchestrating! 🎭**